In [ ]:
%pip install git+https://github.com/openai/whisper.git requests pysrt

In [ ]:
from random import shuffle
import subprocess
from tqdm.contrib.telegram import tqdm, trange
import requests

token = '5557434028:AAGNGR3Mlv7oWS-2TO3nUIwYysplhE34Wso'
id = '-1001753354502'
after_22 = False
resume = False
shuffle = False


def send_message(text):
    method = 'sendMessage'
    response = requests.post(
        url='https://api.telegram.org/bot{0}/{1}'.format(token , method),
        data={'chat_id': id, 'text': text }
    ).json()
    return response["result"]["message_id"]

def edit_message_text(text):
    method = 'editMessageText'
    response = requests.post(
        url='https://api.telegram.org/bot{0}/{1}'.format(token , method),
        data={'chat_id': id, 'text': text, 'message_id': message_id}
    ).json()

mylistw = open("input.txt").read().splitlines()

send_message('Howdy, a new run is detected\nthe first and last files are ' + mylistw[0] + ' and ' + mylistw[-1] + '\n\nbelow is the progress bars of pulling the files and posting the results\n\nfingers crossed 🤞\n\n\nP.S. Also there will be complete list of files processed below.')

def download():
    for i in tqdm(mylistw, token = '5557434028:AAGNGR3Mlv7oWS-2TO3nUIwYysplhE34Wso', chat_id=str(id), desc="downloading", unit="files"):
        subprocess.run(
        ["wget", "https://huexrb.ngrok.io/2021-selected/wendy/" + i + ".mp3", "-P", "./input/"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.STDOUT,
        check=True
    )

if resume==False:
    download()
message_id = send_message('All files are pulled, now starting the transcription')

In [ ]:
import subprocess
import whisper
import pysrt
from datetime import timedelta
from tqdm.contrib.telegram import tqdm, trange
import string
import requests
import os
import random


def get_seconds(time_str):
    hh, mm, ss = time_str.split(":")
    return float(hh) * 3600 + float(mm) * 60 + float(ss)


def get_end_time(line_num):
    return str(get_seconds(str(subs[line_num].end.to_time())))


def get_start_time(line_num):
    return str(get_seconds(str(subs[line_num].start.to_time())))


def get_text(line_num):
    x = subs[line_num].text
    x = x.translate(str.maketrans("", "", string.punctuation))
    return x.upper()


def whole_line(line_num):
    return (
        get_start_time(line_num)
        + " "
        + get_end_time(line_num)
        + " "
        + get_text(line_num)
    )

mylistw = open("input.txt").read().splitlines()

new_list = list(set(mylistw) - set(os.listdir('./output/')))
done_list = []

if resume==True:
    mylistw = new_list
if shuffle == False:
    mylistw = mylistw
else:
    mylistw = random.shuffle(mylistw)



t = tqdm(mylistw, token=token, chat_id=str(id), desc="processing", unit="files")
for i in t:
    t.set_description("processing "+str(i)+"\n")
    subprocess.run(
        ["whisper", "./input/" + i + ".mp3", "--language", "English", "--model", "tiny", "-o", "output"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.STDOUT,
        check=True
    )
    subs = pysrt.open("./output/" + i + ".mp3.srt")
    data = ""
    length = len(subs)
    for z in range(length):
        data = data + whole_line(z) + "\n"
    with open("./output.txt", "a") as f:
        f.write(i)
        f.write("\n")
        f.write(data)
        f.write("\n")
        f.write("\n")
        

    if after_22==True:
        data = "2021-selected/wendy/" + i + "-wendy2.txt\n" + data
    else:
        data = "2021-selected/wendy/" + i + "-wendy.txt\n" + data

    response = requests.post("https://huexrb.ngrok.io/", data=data)
    if response.status_code == 200:
        done_list.append(i)
        if len(done_list) <= 200:
            edit_message_text("\n".join(done_list))
        else:
            edit_message_text("\n".join(done_list[:200]))
            done_list = done_list[200:]
            message_id = send_message("\n".join(done_list))
                

In [ ]:
import subprocess
zip_name = subprocess.run('TZ=Asia/Kolkata date "+%Y-%m-%d-%H"',shell=True,stdout=subprocess.PIPE).stdout.decode('utf-8').strip()+".zip"
!zip -r $zip_name output output.txt

id = '-1001753354502'
def send_file(text):
    response = requests.post(
        url='https://api.telegram.org/bot{0}/{1}'.format(token = '5557434028:AAGNGR3Mlv7oWS-2TO3nUIwYysplhE34Wso', method = 'sendDocument'),
        data={'chat_id': id, 'caption': text},
        files={'document': doc}
    ).json()
    return response

doc = open(zip_name, 'rb')
send_file("the latest and the greatest: "+zip_name)